In [10]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 2000) & (data['year'] <= 2015)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share
data['t']=data['labsh']


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['tg']=data['g']*data['a']
data['cp']=data['g']-data['tg']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean',
                                       'a': "mean",
                                      't': "mean",
                                      'tg':"mean",
                                      'cp':"mean",})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']
summary['TFP Share']=summary['t']
summary['TFP Grows']=summary['tg']
summary['Capital Deeping']=summary['cp']

# Print output
print(summary)

                    g         a         t        tg        cp  Growth Rate  \
countrycode                                                                  
CAN          0.320693  0.387918  0.612082  0.126036  0.194657     0.320693   
DEU          0.595897  0.384552  0.615448  0.236193  0.359704     0.595897   
FRA          0.645835  0.377839  0.622161  0.245273  0.400562     0.645835   
GBR          0.819356  0.375162  0.624838  0.304914  0.514442     0.819356   
ITA         -0.352076  0.470552  0.529448 -0.164108 -0.187968    -0.352076   
JPN          0.752177  0.392263  0.607737  0.298244  0.453933     0.752177   
USA          1.256881  0.387017  0.612983  0.484742  0.772139     1.256881   

             Capital Share  TFP Share  TFP Grows  Capital Deeping  
countrycode                                                        
CAN               0.387918   0.612082   0.126036         0.194657  
DEU               0.384552   0.615448   0.236193         0.359704  
FRA               0.37783